## Quest
- 업무분장(전처리, 모델학습).
- RecurrenceOfSurgery.csv 사용
- 목표변수 : 범주형, 설명변수 : 최소 6개 
- 서비스 대상과 목표 설명, 변수 선택 이유

### 변수설정
0. 서비스 대상: 환자
    * 수술실패에 영향을 미치는 요인들이 무엇이 있을까?
1. target: 수술실패여부
2. features 
    * 범주형: 고혈압여부
    * 범주형: 성별
    * 범주형: 신부전여부
    * 연속형: 연령
    * 연속형: 체중
    * 연속형: 수술시간

In [38]:
import pandas as pd

In [39]:
df_ROS = pd.read_csv('../datasets/RecurrenceOfSurgery.csv')
df_ROS[:2]

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
0,0,1PT,22.8,3,51.0,0.0,0,0,0,0,...,3,51.6,36.6,14.4,0,-1.01,2048.5,4,Down,0
1,1,2PT,44.9,4,26.0,0.0,0,0,0,0,...,0,40.8,7.2,17.8,0,-1.14,1753.1,4,Up,0


In [40]:
df_ROS['수술시간'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1894 entries, 0 to 1893
Series name: 수술시간
Non-Null Count  Dtype  
--------------  -----  
1840 non-null   float64
dtypes: float64(1)
memory usage: 14.9 KB


#### 전처리

In [41]:
df_ROS_select = df_ROS[['수술실패여부', '고혈압여부', '성별', '신부전여부', '연령', '체중', '수술시간']]
df_ROS_select[:2]

,수술실패여부,고혈압여부,성별,신부전여부,연령,체중,수술시간
0,0,0,2,0,66,60.3,68.0
1,0,0,1,0,47,71.7,31.0


#### Scaling & Encoding & Concat

##### - OneHotEncoding

In [42]:
# 범주형 데이터 확인 : '고혈압여부', '성별', '신부전여부'
df_ROS_select['고혈압여부'].value_counts(),df_ROS_select['성별'].value_counts(),df_ROS_select['신부전여부'].value_counts()

(0    1646
 1     248
 Name: 고혈압여부, dtype: int64,
 1    1168
 2     726
 Name: 성별, dtype: int64,
 0    1846
 1      48
 Name: 신부전여부, dtype: int64)

In [43]:
from sklearn.preprocessing import OneHotEncoder

In [44]:
# 범주형 설명변수 OneHotEncoding
oneHotEncoder = OneHotEncoder() # 인스턴스화
oneHotEncoder =oneHotEncoder.fit(df_ROS_select[['고혈압여부', '성별', '신부전여부']])
oneHotEncoder

OneHotEncoder()

In [45]:
oneHotEncoder.categories_

[array([0, 1], dtype=int64),
 array([1, 2], dtype=int64),
 array([0, 1], dtype=int64)]

In [46]:
encoded_data = oneHotEncoder.transform(df_ROS_select[['고혈압여부', '성별', '신부전여부']]).toarray()
encoded_data.shape

(1894, 6)

In [47]:
df_encoded_data = pd.DataFrame(data=encoded_data, columns=oneHotEncoder.get_feature_names_out(['고혈압여부', '성별', '신부전여부']))
df_encoded_data[:2]

,고혈압여부_0,고혈압여부_1,성별_1,성별_2,신부전여부_0,신부전여부_1
0,1.0,0.0,0.0,1.0,1.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0


##### - 병합(Concat)

In [48]:
df_ROS_select= pd.concat([df_ROS_select.reset_index(drop=True), df_encoded_data.reset_index(drop=True)], axis=1)
df_ROS_select[:2]

,수술실패여부,고혈압여부,성별,신부전여부,연령,체중,수술시간,고혈압여부_0,고혈압여부_1,성별_1,성별_2,신부전여부_0,신부전여부_1
0,0,0,2,0,66,60.3,68.0,1.0,0.0,0.0,1.0,1.0,0.0
1,0,0,1,0,47,71.7,31.0,1.0,0.0,1.0,0.0,1.0,0.0


In [49]:
df_ROS_select.shape

(1894, 13)

##### - Scaling

In [50]:
df_ROS_select.columns

Index(['수술실패여부', '고혈압여부', '성별', '신부전여부', '연령', '체중', '수술시간', '고혈압여부_0',
       '고혈압여부_1', '성별_1', '성별_2', '신부전여부_0', '신부전여부_1'],
      dtype='object')

In [51]:
target = df_ROS_select['수술실패여부']
features = df_ROS_select.drop(columns=['수술실패여부', '고혈압여부', '성별', '신부전여부'])

In [52]:
features.columns

Index(['연령', '체중', '수술시간', '고혈압여부_0', '고혈압여부_1', '성별_1', '성별_2', '신부전여부_0',
       '신부전여부_1'],
      dtype='object')

##### - MinMaxScaler

In [53]:
from sklearn.preprocessing import MinMaxScaler

In [71]:
minMaxScaler = MinMaxScaler() #인스턴스화
# features = minMaxSclaer.fit_transform(features)
fit_scaler = minMaxScaler.fit(features)
features_scaler = fit_scaler.transform(features)
features_scaler.shape

(1894, 9)

#### 모델학습, apply()함수를 사용하여 null값 채우기
- null 값 채우기 : '수술시간'

In [55]:
# null값 확인 -> 수술시간 null값 존재
df_ROS_select.isnull().sum()

수술실패여부      0
고혈압여부       0
성별          0
신부전여부       0
연령          0
체중          0
수술시간       54
고혈압여부_0     0
고혈압여부_1     0
성별_1        0
성별_2        0
신부전여부_0     0
신부전여부_1     0
dtype: int64

In [56]:
# null값 삭제 -> null값이 없는 데이터로 모델 학습시키기 위함
df_ROS_select_drop = df_ROS_select.dropna()
df_ROS_select_drop.isnull().sum()

수술실패여부     0
고혈압여부      0
성별         0
신부전여부      0
연령         0
체중         0
수술시간       0
고혈압여부_0    0
고혈압여부_1    0
성별_1       0
성별_2       0
신부전여부_0    0
신부전여부_1    0
dtype: int64

In [57]:
# null값이 없는 데이터로 모델학습 준비
# 1. target : '수술시간', feature: '성별'
target = df_ROS_select_drop[['수술시간']]
feature = df_ROS_select_drop[['성별']]
target.shape, feature.shape

((1840, 1), (1840, 1))

In [58]:
# 2. null값이 없는 데이터와 실제값을 사용하여 회귀모델 훈련
from sklearn.linear_model import LinearRegression
model = LinearRegression() # 인스턴스화(초기화)
model.fit(feature, target) # 모델 훈련

LinearRegression()

In [59]:
# 모델 예측 확인해보기 (type : numpy의 array)
model_ = model.predict(feature)

In [60]:
model_list = model_.tolist()
model_list[:2]

[[62.407988587731815], [61.856014047410014]]

In [61]:
# apply()
import numpy as np
def convert_notnull(row) :
    if np.isnan(row['수술시간'])  : # 변수 row의 값이 null이라면
        feature = df_ROS_select[['성별']]
        result = model.predict(feature)
        return result[0]
    else :
        return row['수술시간']  # null이 아니면 원래 데이터 값 반환

In [62]:
df_ROS_select['수술시간'] = df_ROS_select.apply(convert_notnull, axis=1)
df_ROS_select['수술시간']

0       68.0
1       31.0
2       78.0
3       73.0
4       29.0
        ... 
1889    80.0
1890    20.0
1891    50.0
1892    25.0
1893    45.0
Name: 수술시간, Length: 1894, dtype: object

In [63]:
# apply() 적용 후 null값 확인
df_ROS_select['수술시간'].isnull().sum()

0

In [64]:
df_ROS_select.isnull().sum()

수술실패여부     0
고혈압여부      0
성별         0
신부전여부      0
연령         0
체중         0
수술시간       0
고혈압여부_0    0
고혈압여부_1    0
성별_1       0
성별_2       0
신부전여부_0    0
신부전여부_1    0
dtype: int64

#### best model 찾기
- hypertuning
- resampling

In [73]:
target = df_ROS_select['수술실패여부']
features = df_ROS_select.drop(columns=['수술실패여부', '고혈압여부', '성별', '신부전여부'])
df_ROS_select.drop(columns=['수술실패여부', '고혈압여부', '성별', '신부전여부']).isnull().sum()

연령         0
체중         0
수술시간       0
고혈압여부_0    0
고혈압여부_1    0
성별_1       0
성별_2       0
신부전여부_0    0
신부전여부_1    0
dtype: int64

##### - HyperTuning

In [76]:
# resampling 하기 전
from collections import Counter
Counter(df_ROS_select['수술실패여부'])

Counter({0: 1779, 1: 115})

In [77]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
scoring = make_scorer(f1_score)

In [78]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()

In [79]:
hyper_list = {'max_depth':range(2, 10),
              'min_samples_leaf': range(2, 10),
              'criterion': ['gini','entropy'],
              'class_weight': [None, 'balanced'],
              'min_samples_split': range(2, 10)}
grid_model = GridSearchCV(model, param_grid=hyper_list,
                          scoring=scoring,
                          n_jobs = -1,
                          cv = 5)

##### - resampling
- under sampling : NearMiss

In [80]:
df_ROS_select.isnull().sum()

수술실패여부     0
고혈압여부      0
성별         0
신부전여부      0
연령         0
체중         0
수술시간       0
고혈압여부_0    0
고혈압여부_1    0
성별_1       0
성별_2       0
신부전여부_0    0
신부전여부_1    0
dtype: int64

In [85]:
from imblearn.under_sampling import NearMiss
from datetime import datetime

In [86]:
sampling_strategy_list = np.arange(0.3, 1.0, 0.05)

for sampling_strategy in sampling_strategy_list :
    print(30*'-','{}'.format(sampling_strategy))
    resampler = NearMiss(sampling_strategy=sampling_strategy)
    features_under_resample, target_under_resample = resampler.fit_resample(features, target)
    print(Counter(target_under_resample))
    features_under_resample_train, features_under_resample_test, target_under_resample_train, target_under_resample_test = train_test_split(features_under_resample, target_under_resample, test_size = 0.3, random_state= 1234)
    print(Counter(target_under_resample_train), Counter(target_under_resample_test))
    # re learning with oversampling datasets
    
    start = datetime.now()  # start date and time

    grid_model.fit(features_under_resample_train, target_under_resample_train)

    end = datetime.now()  # end date and time - set to current date and time

    duration = end - start  # duration as a timedelta object

    print("Duration: {0}".format(duration))
    
    best_model = grid_model.best_estimator_
    target_under_resample_train_pred = best_model.predict(features_under_resample_train)
    target_under_resample_test_pred = best_model.predict(features_under_resample_test)

    print(classification_report(target_under_resample_train, target_under_resample_train_pred))
    print(classification_report(target_under_resample_test, target_under_resample_test_pred))
    print(30*'-')

------------------------------ 0.3
Counter({0: 383, 1: 115})


NameError: name 'train_test_split' is not defined